# High-Performance Pandas: eval() and query()

As we´ve already seen in previous sections, the power of the PyData stack is built upong the ability of NumPy and Pandas to push basic operations into C via and intuitive syntax: examples are vectorized/broadcasted operations in NumPy, and grouping-type operations in Pandas. While these abstractions are efficient and effective for many common use cases, they often rely on the creation of temporary intermediate objects, which can cause undue overhead in computational time and memory use.

Pandas includes some experimental tools that allow you to directly access C-speed operations without costly allocation of intermediate arrays. These are the `eval()` and `query()` functions, which rely on the [Numexpr](https://github.com/pydata/numexpr) package. In this notebook we will walk through their use and give some rules-of-thumb about when you might think about using them.

## Motivating `query()` and `eval()`: Compound Expressions
We've seen previously that NumPy and Pandas support fast vectorized operations; for example, when adding the elements of two arrays:

In [1]:
import numpy as np 
rng = np.random.RandomState(4)
x = rng.rand(1_000_000)
y = rng.rand(1_000_000)
%timeit x + y

9.42 ms ± 680 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


As discussed, this is much faster than doing the addition via a Python loop or comprehension:

In [2]:
%timeit np.fromiter((xi + yi for xi, yi in zip(x, y)), dtype=x.dtype, count=len(x))

608 ms ± 148 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


But this abstraction can become less efficient when computing compound expressions. For example, consider the following expression:

In [3]:
mask = (x > 0.5) & (y < 0.5)

Because NumPy evaluates each subexpression, this is roughly equivalent to the following:

In [4]:
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2

In other words, *every intermediate step is explicitly allocated in memory*. If the `x` and `y` arrays are very large, this can lead to significant memory and computational overhead. The Numexpr library gives you the ability to compute this type of compound expression element by element, without the need to allocate full intermediate a arrays. The Numexpr documentation has more details, but for the time being it is sufficient to say that the library accepts a *string* giving the NumPy-style expression you'd like to compute:

In [6]:
import numexpr
mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
np.allclose(mask, mask_numexpr)

True

The benefit here is that Numexpr evaluates the expression in a way that does not use full-size temporary arrays, and thus can be much more efficient than NumPy, especially for large arrays. The Pandas `eval()` and `query()` tools that we will discuss here are conceptually similar, and depend on the Numexpr package.

## `pandas.eval()` for Efficient Operations

The `eval()` function in Pandas uses string expressions to efficiently compute operations using `DataFrame`s. For example, consider the following `DataFrame`s:

In [7]:
import pandas as pd 
nrows, ncols = 100_000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols)) for i in range(4))

To compute hte sum of all four `DataFrame`s using the typical pandas approach, we can just write the sum:

In [8]:
%timeit df1 + df2 + df3 + df4

101 ms ± 15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%timeit pd.eval('df1 + df2 + df3 + df4')

34.8 ms ± 929 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


The `eval()` version of this expression is about 50% faster (and uses much less memory), while giving the same result:

In [10]:
np.allclose(df1 + df2 + df3 + df4, pd.eval('df1 + df2 + df3 + df4'))

True

### Operations supported by `pd.eval()`
As of Pandas v0.16, `pd.eval()` supports a wide range of oeprations. To demostrate these, we'll use the following integer `DataFrame`s:

In [11]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 100, (100, 3))) for i in range(5))

#### Arithmetic operators
`pd.eval()` supports all arithmetic operators. For example:


In [14]:
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

#### Comparison operators
`pd.eval()` supports all comparison operators, including chained expressions:

In [16]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

#### Bitwise operators
`pd.eval()` supports the & and | bitwise operators: 

In [17]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

In addition, it supports the use of the literal `and` and `or` in Boolean expressions:

In [19]:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

#### Object attributes and indices
`pd.eval()` supports access to object attributes via the `obj.attr` syntax, and indexes via the `obj[index]` syntax:

In [20]:
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

#### Other operations
Other operations such as function calls, conditional statements, loops, and other more involved construct are currently *not* implemented in `pd.eval()`. If you'd like to execute these more complicated types of expressions, you can use the Numexpr library itself.